![Logo](images/rosdevday2023_logo.png)

# Build and use a Mobile Manipulator with ros2_control #

[ros2_control](https://control.ros.org) is a hardware-agnostic control framework for abstracting hardware and low-level control for 3rd party solutions like `MoveIt2` and `Nav2` systems.

The rosject will start with a quick recap of what makes up a URDF and the ros2_control URDF tag. 
We’ll take a robot mobile base and demonstrate setting up ros2_control simulation for it in the URDF. 

Once done, a robot arm will be added to the mobile base, turning it into a mobile manipulator robot. 
The existing ros2_control configuration will be adjusted to accommodate for the new robot parts. 
Finally, the rosject will conclude with a demonstration in Gazebo moving the robot using some off-the-shelf ros2_control controllers.

Attendees will learn
* how to prototype a mobile manipulator with URDF and `ros2_control`
* how to configure a gazebo simulation with a given URDF and `ros2_control`

This project is using a simulated [TIAGo robot from PAL Robotics](https://pal-robotics.com/robots/tiago/) for the demonstration. 

We'll go from a mobile base to a full robot!

![TIAGo by PAL Robotics](images/tiago_parts.png)

## WHO AM I

Bence Magyar, PhD in Robotics, Principal Software Engineer at FiveAI / BoschUK

Maintainer of ros_control and ros2_control since 2015.
ROS contributor since 2012.

<img src="images/bence.jpeg" width="30%">



# URDF crashcourse

Unified Robotics Description Format (URDF), is an XML specification used in academia and industry to model multibody systems such as robotic manipulator arms for manufacturing assembly lines and animatronic robots for amusement parks. URDF is especially popular with users of Robotics Operating System, ROS.

<img src="images/tiago-tf.png" width="100%">


# The URDF of a differential wheeled robot

Let's open the URDF for the TIAGo-Base and examine the structure.

Go to the code editor and find `tiago_robot/tiago_description/robots/tiago_base_unrolled.urdf`

#### Let's take a look

In [ ]:
source ~/ros2_ws/install/setup.bash
alias s="source ~/ros2_ws/install/setup.bash"
ros2 launch tiago_description show.launch.py

# ros2_control crashcourse

ros2_control is a hardware-agnostic control framework focusing on the modular composition of control systems for robots, sharing of controllers as well as real-time performance. The framework provides controller-lifecycle and hardware management on top of abstractions of real or virtual hardware interfaces.

<img src="images/ros2_control_overview.png" width="100%">


<img src="images/components_architecture.png" width="80%">


# The ros2_control tag for URDF

Covers both simulation and real hardware components.
Example:
```xml
<ros2_control name="ros2_control_tiago_system" type="system">
    <hardware>
        <plugin>gazebo_ros2_control/GazeboSystem</plugin>
    </hardware>
    <joint name="wheel_right_joint">
        <command_interface name="velocity">
            <param name="min">-1</param>
            <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
    </joint>
    <transmission name="wheel_right_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="wheel_right_actuator" role="actuator1"/>
        <joint name="wheel_right_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <joint name="wheel_left_joint">
        <command_interface name="velocity">
            <param name="min">-1</param>
            <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
    </joint>
    <transmission name="wheel_left_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="wheel_left_actuator" role="actuator1"/>
        <joint name="wheel_left_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <sensor name="base_imu_sensor">
        <state_interface name="orientation.x"/>
        <state_interface name="orientation.y"/>
        <state_interface name="orientation.z"/>
        <state_interface name="orientation.w"/>
        <state_interface name="angular_velocity.x"/>
        <state_interface name="angular_velocity.y"/>
        <state_interface name="angular_velocity.z"/>
        <state_interface name="linear_acceleration.x"/>
        <state_interface name="linear_acceleration.y"/>
        <state_interface name="linear_acceleration.z"/>
    </sensor>
</ros2_control>
```

# Launch the simultation and examine the system

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_gazebo tiago_gazebo.launch.py 

In a different terminal, let's check the hardware drivers:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_hardware_components

```
Hardware Component 0
	name: ros2_control_tiago_system
	type: 
	plugin name: plugin name missing!
	state: id=3 label=active
	command interfaces
		wheel_right_joint/velocity [available] [unclaimed]
		wheel_left_joint/velocity [available] [unclaimed]
```

Let's check the available controllers:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_controllers

```
joint_state_broadcaster[joint_state_broadcaster/JointStateBroadcaster] active
mobile_base_controller[diff_drive_controller/DiffDriveController] unconfigured
```

Note that the `mobile_base_controller` is launched (see `mobile_base_controller.launch.py`), however it errors out due to not having the right configuration.

(Highlighted here for posterity)
```
[gzserver-1] [ERROR] [1686512444.304216553] [mobile_base_controller]: Wheel names parameters are empty!
[gzserver-1] [WARN] [1686512444.305330650] []: Error occurred while doing error handling.
[gzserver-1] [ERROR] [1686512444.305499086] [controller_manager]: After configuring, controller'mobile_base_controller' is in state 'unconfigured' , expected inactive.
```

Let's fix that!


Open `mobile_base_controller.yaml` and add these lines:

```
    use_stamped_vel: False
    left_wheel_names  : [???]
    right_wheel_names : [???]

    wheel_radius: 0.0985
    wheel_separation: 0.4044

    wheel_separation_multiplier: 1.0
    left_wheel_radius_multiplier: 1.0
    right_wheel_radius_multiplier: 1.0
```
Mystery: what goes into the values marked with `???`

Solved? Let's run it all again.

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_gazebo tiago_gazebo.launch.py 

And as before, let's check the hardware drivers:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_hardware_components

```
Hardware Component 0
	name: ros2_control_tiago_system
	type: 
	plugin name: plugin name missing!
	state: id=3 label=active
	command interfaces
		wheel_right_joint/velocity [available] [claimed]
		wheel_left_joint/velocity [available] [claimed]
```

Note that the wheel joint command interfaces are now claimed, something is happening!
Let's list the available interfaces controllers can use:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_hardware_interfaces


```
command interfaces
	wheel_left_joint/velocity [available] [claimed]
	wheel_right_joint/velocity [available] [claimed]
state interfaces
	base_imu_sensor/angular_velocity.x
	base_imu_sensor/angular_velocity.y
	base_imu_sensor/angular_velocity.z
	base_imu_sensor/linear_acceleration.x
	base_imu_sensor/linear_acceleration.y
	base_imu_sensor/linear_acceleration.z
	base_imu_sensor/orientation.w
	base_imu_sensor/orientation.x
	base_imu_sensor/orientation.y
	base_imu_sensor/orientation.z
	wheel_left_joint/position
	wheel_right_joint/position
```

Listing controllers again should yield an active base controller:


In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_controllers

```
joint_state_broadcaster[joint_state_broadcaster/JointStateBroadcaster] active      
mobile_base_controller[diff_drive_controller/DiffDriveController] active    
```

Let's check a controller, they are ROS Nodes after all!

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 node info /mobile_base_controller 

```
/mobile_base_controller
  Subscribers:
    /clock: rosgraph_msgs/msg/Clock
    /mobile_base_controller/cmd_vel_unstamped: geometry_msgs/msg/Twist
    /parameter_events: rcl_interfaces/msg/ParameterEvent
  Publishers:
    /mobile_base_controller/odom: nav_msgs/msg/Odometry
    /mobile_base_controller/transition_event: lifecycle_msgs/msg/TransitionEvent
    /parameter_events: rcl_interfaces/msg/ParameterEvent
    /rosout: rcl_interfaces/msg/Log
    /tf: tf2_msgs/msg/TFMessage
  Service Servers:
    /mobile_base_controller/describe_parameters: rcl_interfaces/srv/DescribeParameters
    /mobile_base_controller/get_parameter_types: rcl_interfaces/srv/GetParameterTypes
    /mobile_base_controller/get_parameters: rcl_interfaces/srv/GetParameters
    /mobile_base_controller/list_parameters: rcl_interfaces/srv/ListParameters
    /mobile_base_controller/set_parameters: rcl_interfaces/srv/SetParameters
    /mobile_base_controller/set_parameters_atomically: rcl_interfaces/srv/SetParametersAtomically
  Service Clients:

  Action Servers:

  Action Clients:
```

You can make the robot drive in the simulation with this:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 topic pub /mobile_base_controller/cmd_vel_unstamped geometry_msgs/msg/Twist '{linear: {x: 1}, angular: {z: 0}}' -r10

# The URDF of a mobile manipulator

Let's open the URDF for the TIAGo-Base and examine the structure.

Go to the code editor and find `tiago_robot/tiago_description/robots/tiago_full_unrolled.urdf`

```xml
<ros2_control name="ros2_control_tiago_system" type="system">
    <hardware>
        <plugin>gazebo_ros2_control/GazeboSystem</plugin>
    </hardware>
    <joint name="wheel_right_joint">
        <command_interface name="velocity">
            <param name="min">-1</param>
            <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
    </joint>
    <transmission name="wheel_right_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="wheel_right_actuator" role="actuator1"/>
        <joint name="wheel_right_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <joint name="wheel_left_joint">
        <command_interface name="velocity">
            <param name="min">-1</param>
            <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
    </joint>
    <transmission name="wheel_left_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="wheel_left_actuator" role="actuator1"/>
        <joint name="wheel_left_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <sensor name="base_imu_sensor">
        <state_interface name="orientation.x"/>
        <state_interface name="orientation.y"/>
        <state_interface name="orientation.z"/>
        <state_interface name="orientation.w"/>
        <state_interface name="angular_velocity.x"/>
        <state_interface name="angular_velocity.y"/>
        <state_interface name="angular_velocity.z"/>
        <state_interface name="linear_acceleration.x"/>
        <state_interface name="linear_acceleration.y"/>
        <state_interface name="linear_acceleration.z"/>
    </sensor>
    <joint name="torso_lift_joint">
        <command_interface name="position"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="torso_lift_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="torso_lift_actuator" role="actuator1"/>
        <joint name="torso_lift_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="head_1_joint">
        <command_interface name="position"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="head_1_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="head_1_actuator" role="actuator1"/>
        <joint name="head_1_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="head_2_joint">
        <command_interface name="position"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="head_2_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="head_2_actuator" role="actuator1"/>
        <joint name="head_2_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_1_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_1_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="arm_1_actuator" role="actuator1"/>
        <joint name="arm_1_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_2_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_2_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="arm_2_actuator" role="actuator1"/>
        <joint name="arm_2_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_3_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_3_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="arm_3_actuator" role="actuator1"/>
        <joint name="arm_3_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_4_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_4_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="arm_4_actuator" role="actuator1"/>
        <joint name="arm_4_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_5_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_5_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="arm_5_actuator" role="actuator1"/>
        <joint name="arm_5_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="arm_6_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <joint name="arm_7_joint">
        <command_interface name="position"/>
        <command_interface name="velocity"/>
        <command_interface name="effort"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="arm_6_arm_7_wrist_trans">
        <plugin>transmission_interface/DifferentialTransmission</plugin>
        <actuator name="arm_6_actuator" role="actuator1">
            <mechanical_reduction>-1.0</mechanical_reduction>
        </actuator>
        <actuator name="arm_7_actuator" role="actuator2">
            <mechanical_reduction>1.0</mechanical_reduction>
        </actuator>
        <joint name="arm_6_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
        <joint name="arm_7_joint" role="joint2">
            <mechanical_reduction>1.0</mechanical_reduction>
            <offset>0.0</offset>
        </joint>
    </transmission>
    <joint name="gripper_left_finger_joint">
        <command_interface name="position"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="gripper_left_finger_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="gripper_left_finger_actuator" role="actuator1"/>
        <joint name="gripper_left_finger_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <joint name="gripper_right_finger_joint">
        <command_interface name="position"/>
        <state_interface name="position"/>
        <state_interface name="velocity"/>
        <state_interface name="effort"/>
    </joint>
    <transmission name="gripper_right_finger_trans">
        <plugin>transmission_interface/SimpleTransmission</plugin>
        <actuator name="gripper_right_finger_actuator" role="actuator1"/>
        <joint name="gripper_right_finger_joint" role="joint1">
            <mechanical_reduction>1.0</mechanical_reduction>
        </joint>
    </transmission>
    <sensor name="wrist_ft_joint_force_torque">
        <state_interface name="force.x"/>
        <state_interface name="force.y"/>
        <state_interface name="force.z"/>
        <state_interface name="torque.x"/>
        <state_interface name="torque.y"/>
        <state_interface name="torque.z"/>
    </sensor>
</ros2_control>
```

#### Let's take a look

Edit the file `tiago_robot/tiago_description/launch/robot_state_publisher.launch.py` at line 50. Comment `tiago_base_unrolled` and comment out `tiago_full_unrolled` to get the full robot.

To visualize what we got:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_description show.launch.py

## Launch the simulation and examine the system (again)

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_gazebo tiago_gazebo.launch.py

And do a quick examination in a different terminal:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_hardware_interfaces

```
command interfaces
        arm_1_joint/effort [available] [unclaimed]
        arm_1_joint/position [available] [unclaimed]
        arm_1_joint/velocity [available] [unclaimed]
        arm_2_joint/effort [available] [unclaimed]
        arm_2_joint/position [available] [unclaimed]
        arm_2_joint/velocity [available] [unclaimed]
...
        gripper_left_finger_joint/position [available] [unclaimed]
        gripper_right_finger_joint/position [available] [unclaimed]
        head_1_joint/position [available] [unclaimed]
        head_2_joint/position [available] [unclaimed]
        torso_lift_joint/position [available] [unclaimed]
        wheel_left_joint/velocity [available] [claimed]
        wheel_right_joint/velocity [available] [claimed]
state interfaces
        arm_1_joint/effort
        arm_1_joint/position
        arm_1_joint/velocity
        arm_2_joint/effort
        arm_2_joint/position
        arm_2_joint/velocity
...
```

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_controllers

```
mobile_base_controller[diff_drive_controller/DiffDriveController] active
joint_state_broadcaster[joint_state_broadcaster/JointStateBroadcaster] active
```

Oh no, we need controllers!
Let's add them to the launch commands in `tiago_controller_configuration/launch/default_controllers.launch.py`.
Open this file and uncomment lines 72-77.


Let's relaunch the simulation and check on those controllers!

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_gazebo tiago_gazebo.launch.py

...and in a different terminal:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_controllers

```
gripper_controller  [joint_trajectory_controller/JointTrajectoryController] active
joint_state_broadcaster[joint_state_broadcaster/JointStateBroadcaster] active
head_controller     [joint_trajectory_controller/JointTrajectoryController] active
mobile_base_controller[diff_drive_controller/DiffDriveController] active
arm_controller      [joint_trajectory_controller/JointTrajectoryController] unconfigured
torso_controller    [joint_trajectory_controller/JointTrajectoryController] active
```

### Help! I forgot to add the configuration for the arm controller!

We'll need to work on the configuration file of the arm controller in `tiago_robot/tiago_controller_configuration/config/arm_controller.yaml`.

So far it looks like this
```
arm_controller:
  ros__parameters:
    type: joint_trajectory_controller/JointTrajectoryController
```
We need to add some more content!
```
    joints:
      - ...
    command_interfaces:
      - ...
    state_interfaces:
      - ...
```
### Let's work out what goes in place of the `...` sections


for good measure, let's add some constraints:
```
    constraints:
      goal_time: 0.6
      stopped_velocity_tolerance: 5.0
      arm_1_joint:
        goal: 0.02
      arm_2_joint:
        goal: 0.02
      arm_3_joint:
        goal: 0.02
      arm_4_joint:
        goal: 0.02
      arm_5_joint:
        goal: 0.02
      arm_6_joint:
        goal: 0.02
      arm_7_joint:
        goal: 0.02
```

Now you are all pros, let's run the following to verify that all controllers are up and running & joint interfaces are claimed, etc:


In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 launch tiago_gazebo tiago_gazebo.launch.py

...and in a different terminal:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 control list_hardware_components
ros2 control list_hardware_interfaces
ros2 control list_controllers

Let's move the arm this time:

In [ ]:
source ~/ros2_ws/install/setup.bash
ros2 topic pub /torso_controller/joint_trajectory trajectory_msgs/msg/JointTrajectory "{joint_names: ["torso_lift_joint"],points: [{positions: [0.5], time_from_start: {sec: 1.0}}] }"

Hit enter and watch the robot move in Gazebo!

# References

* [control.ros.org](https://control.ros.org)
* Main repositories [ros2_control](https://github.com/ros-controls/ros2_control), [ros2_controllers](https://github.com/ros-controls/ros2_controllers), [ros2_control_demos](https://github.com/ros-controls/ros2_control_demos)
* [ROS wiki URDF page](http://wiki.ros.org/urdf)
* [URDF tutorial](https://articulatedrobotics.xyz/ready-for-ros-7-urdf/) by Articulated Robotics

![headerlogos](images/header_logos.png)